# 融合多个模型的预测结果
通过 IOU 判断同一个 bbox

In [185]:
import pandas as pd
import os
import numpy as np
import sys

In [186]:
min_size = 12 # 12 最佳
iou_threshold = 0.7
path = '../data/summit/'
# path = '../data/summi_huo/'
# file_list = ['summit_frir2_027_41097', 'summit_frr101_028_29124'] #过滤bbox < 12 = merged_005 = 0.864368
# file_list = ['merged_005', 'summit_frir2_027_40343'] 
file_list = ['summi_huo-717', 'summit_yolo3_010_final'] 

In [187]:
def IOU(Reframe,GTframe):
    """
    https://blog.csdn.net/eddy_zheng/article/details/52126641
    自定义函数，计算两矩形 IOU，传入为均为矩形坐标和尺寸 [x,y, width, height]
    """
    x1, y1, width1, height1 = Reframe
    x2, y2, width2, height2 = GTframe

    endx = max(x1 + width1, x2 + width2)
    startx = min(x1, x2)
#     计算重叠部分的宽度
    width = width1 + width2 - (endx - startx)

    endy = max(y1 + height1, y2 + height2)
    starty = min(y1, y2)
#     计算重叠部分的高度
    height = height1 + height2 - (endy - starty)

    if width <=0 or height <= 0:
        ratio = 0 # 重叠率为 0 
    else:
        Area = width * height # 两矩形相交面积
        Area1 = width1 * height1
        Area2 = width2 * height2
        ratio = Area/(Area1 + Area2 - Area)
    
    return ratio,Reframe,GTframe

In [188]:
def get_coor_from_str(coor_str):
    bbox = coor_str.split('_')
    b = [int(float(bbox[0])), int(float(bbox[1])), int(float(bbox[2])), int(float(bbox[3]))]
    return b

In [189]:
def merge_bbox(bbox1, bbox2):
    bbox = []
    for i in range(len(bbox1)):
        bbox.append(int(np.round((bbox1[i] + bbox2[i]) / 2)))
    return bbox

In [190]:
def merge_coordinate(coordinate1, coordinate2):
    list1 = coordinate1.split(';')
#     print('list1 =', len(list1))
    list2 = coordinate2.split(';')
#     print('list2 =', len(list2))
#     两个连起来，两两对比
    corr_list = list1+list2
    coor_num = len(corr_list)
#     print('corr_list=', corr_list)
    merged_list = []
    result_list = []
    out_str = ''
    for i in range(0, coor_num):
        bbox1 = get_coor_from_str(corr_list[i])
#         print('%s bbox1=%s'%(i, bbox1))
        if i in merged_list:
#             print('已合并 bbox1=', bbox1)
            continue
        
        if bbox1[2] < min_size or bbox1[3] < min_size:
#             print('忽略太小的 bbox1=', bbox1)
            continue
        bbox = bbox1 #默认没有相同的，取自身 
        for j in range(i+1, coor_num):
            bbox2 = get_coor_from_str(corr_list[j])
#             print('%s bbox2=%s'%(j, bbox2))
            if j in merged_list:
#                 print('已合并 bbox2=', bbox2)
                continue
            if bbox2[2] < min_size or bbox2[3] < min_size:
#                 print('忽略太小的 bbox2=', bbox2)
                continue
            iou = IOU(bbox1, bbox2)
#             print('iou =', iou)
            if iou[0] > iou_threshold:
                bbox = merge_bbox(bbox1, bbox2)
#                 print('有相同的，则合并 bbox=', bbox)
#                 标记为已合并
                merged_list.append(j)
        
        if len(result_list) > 0:
            out_str += ';'
        
        result_list.append(bbox)
        out_str += '_'.join([str(bbox[0]), str(bbox[1]), str(bbox[2]), str(bbox[3])])
#         break
#     print('merged_list =', len(merged_list))
#     print('result_list =', len(result_list))
    return out_str

In [191]:
# info_1 = pd.read_csv('%s%s.csv'%(path, file_list[0]))
# info_1.head()

In [192]:
# info_2 = pd.read_csv('%s%s.csv'%(path, file_list[1]))
# info_2.head()

In [193]:
def merge_info(info_1, info_2):
    image_num = len(info_1)
    result_df = info_1.copy()
    for i in range(0, image_num):
        info = info_1.iloc[i:i+1]
        name = info.name.values[0]
#         print('name1 =', name)
        coordinate = info.coordinate.values[0]
#         print('coordinate =', coordinate)
        info2 = info_2.iloc[i:i+1]
        name2 = info2.name.values[0]
#         print('name2 =', name2)
        coordinate2 = info2.coordinate.values[0]
#         print('coordinate2 =', coordinate2)
        
        result_row = result_df.iloc[i:i+1]
        if not isinstance(coordinate, str):
            coordinate_merged = coordinate2
        elif not isinstance(coordinate2, str):
            coordinate_merged = coordinate
        else:
            coordinate_merged = merge_coordinate(coordinate, coordinate2)
#         print('coordinate_merged =', coordinate_merged)
        result_row.coordinate.values[0] = coordinate_merged
        sys.stdout.write('\r>> i = %s / %s' % ((i+1),image_num))
        sys.stdout.flush()
#         if (i > 0):
#             break
    sys.stdout.write('\n')
    sys.stdout.flush()
    return result_df

# merge_info(info_1, info_2)

In [194]:
for i in range(0, len(file_list)):
    if i == 0:
        info_0 = pd.read_csv('%s%s.csv'%(path, file_list[i]))
    else:
        info_other = pd.read_csv('%s%s.csv'%(path, file_list[i]))
        info_0 = merge_info(info_0, info_other)
info_0.to_csv('%s%s.csv'%(path, 'merged'), index=False)

>> i = 5000 / 5000


In [195]:
info_other.head()

,name,coordinate
0,0000b1da-64b9-4f4b-9291-d75ddc00ea1d.jpg,193_0_59_19;488_256_134_124;207_242_176_185;14...
1,0003be84-3f11-4ddf-b409-cd00a7b78ceb.jpg,440_20_90_81;606_5_109_133;433_99_112_120;460_...
2,000594ea-2b16-4e9a-a58e-f30d956986b4.jpg,379_202_178_186;1_205_37_282;324_5_120_33;0_71...
3,0006a207-6339-4bb6-8d57-75a4e29d3c7f.jpg,188_141_170_208;589_249_179_217;0_167_65_190;8...
4,0008e472-76f6-414a-b334-32a9dcfc30b0.jpg,487_3_36_18;702_7_57_85;617_8_69_83;428_6_77_8...


In [196]:
info_0.head()

,name,coordinate
0,0000b1da-64b9-4f4b-9291-d75ddc00ea1d.jpg,210_241_172_188;140_6_72_50;196_0_57_18;495_23...
1,0003be84-3f11-4ddf-b409-cd00a7b78ceb.jpg,456_392_220_103;229_198_159_170;168_72_142_96;...
2,000594ea-2b16-4e9a-a58e-f30d956986b4.jpg,376_198_184_194;1_74_150_157;385_40_130_114;32...
3,0006a207-6339-4bb6-8d57-75a4e29d3c7f.jpg,590_246_180_218;84_409_232_88;184_143_177_211;...
4,0008e472-76f6-414a-b334-32a9dcfc30b0.jpg,361_192_158_180;428_6_74_80;0_179_59_122;614_6...
